# Classificação e previsão de dados: Valorant

Este projeto se resume a previsão de eliminações ("kills") que um jogador pode fazer no jogo Valorant, segundo seu numero de assistencias, mortes, personagem utilizado e seu KD (Proporção de kills feitas x quantidade de mortes do seu personagem).

Dataset utilizado: [VALORANT Masters 2021: Player & Map Data](https://www.kaggle.com/datasets/jaykumar1607/valorant-masters-2021-player-map-data?rvi=1&select=player_stats.csv)


Um projeto de:
- Guilherme Carneiro De Matos
- Amorgan Mendes Lopes





In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Carregar Dataset
df = pd.read_csv("caminho_do_seu_arquivo")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   50 non-null     object 
 1   Country  50 non-null     object 
 2   Team     50 non-null     object 
 3   Agents   50 non-null     object 
 4   Maps     50 non-null     int64  
 5   K        50 non-null     int64  
 6   D        50 non-null     int64  
 7   A        50 non-null     int64  
 8   KD       50 non-null     float64
 9   KDA      50 non-null     float64
 10  ACS/Map  50 non-null     int64  
 11  K/Map    50 non-null     float64
 12  D/Map    50 non-null     float64
 13  A/Map    50 non-null     float64
dtypes: float64(5), int64(5), object(4)
memory usage: 5.6+ KB


In [ ]:
# Selecionar as features e o target
X = df[['A', 'D', 'Agents', 'KD']]
y = df['K']

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Pipeline do pré-processamento
numeric_features = ['A', 'D', 'KD']
categorical_features = ['Agents']

In [ ]:
# Criar pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Combinar pré-processador com o modelo KNN
knn_regressor_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

In [ ]:
# Combinar pré-processador com o modelo SVM
svm_regressor_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

In [ ]:
# Treinar modelo KNN
knn_regressor_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['A', 'D', 'KD']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Agents'])])),
                ('regressor', KNeighborsRegressor())])

In [ ]:
# Treinar modelo SVM
svm_regressor_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['A', 'D', 'KD']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Agents'])])),
                ('regressor', SVR())])

In [ ]:
# Fazer previsões
y_pred_knn_regressor = knn_regressor_model.predict(X_test)
y_pred_svm_regressor = svm_regressor_model.predict(X_test)


In [ ]:
# Avaliar os modelos
r2_knn_regressor = r2_score(y_test, y_pred_knn_regressor)
r2_svm_regressor = r2_score(y_test, y_pred_svm_regressor)

In [ ]:
# Previsão de kills
example_data = pd.DataFrame([[20, 30, "['Jett']", 1.0]], columns=['A', 'D', 'Agents', 'KD'])
kills_pred = int(round(knn_regressor_model.predict(example_data)[0]))
kills_pred_svm = int(round(svm_regressor_model.predict(example_data)[0]))

In [ ]:
# Exibir resultado
print("Acurácia KNN:", r2_knn_regressor)
print("Acurácia SVM:", r2_svm_regressor)
print("--------------------------------------")
print("Previsão de kills KNN:", kills_pred)
print("Previsão de kills SVM:", kills_pred_svm)

Acurácia KNN: 0.933244043399578
Acurácia SVM: 0.10073564088011544
--------------------------------------
Previsão de kills KNN: 68
Previsão de kills SVM: 110
